In [28]:
import tabula as tb
import pandas as pd

In [36]:
# race
df = tb.read_pdf("../Data/Race/2014-FRA-RAC.pdf", area = (120, 0, 500, 222), columns=[72, 78, 90, 110], pages = "1")[0]
df

Got stderr: Feb 09, 2023 6:12:56 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Feb 09, 2023 6:12:56 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Feb 09, 2023 6:12:56 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Feb 09, 2023 6:12:56 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



,Pos,Unnamed: 0,Unnamed: 1,Unnamed: 2,Rider N
0,1,NaN,25,93,Marc MARQUEZ
1,2,NaN,20,46,Valentino ROSSI
2,3,NaN,16,19,Alvaro BAUTISTA
3,4,NaN,13,44,Pol ESPARGARO
4,5,NaN,11,26,Dani PEDROSA
5,6,NaN,10,99,Jorge LORENZO
6,7,NaN,9,6,Stefan BRADL
7,8,NaN,8,4,Andrea DOVIZIOSO
8,9,NaN,7,41,Aleix ESPARGARO
9,10,NaN,6,38,Bradley SMITH


In [37]:
# remove all instances of 'Pos' that are not an integer


0       1
1       2
2       3
3       4
4       5
5       6
6       7
7       8
8       9
9      10
10     11
11     12
12     13
13     14
14     15
15     16
16     17
17     18
18     19
19    Not
20    NaN
21    NaN
22    Not
23    NaN
24      R
25    NaN
Name: Pos, dtype: object

In [4]:
filename = "../Data/FP/2013-AME-FP4.pdf"

In [34]:
tb.read_pdf(filename, area = (150, 0, 730, 250), pages = "1")[0]

Got stderr: Feb 09, 2023 6:10:02 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Feb 09, 2023 6:10:02 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>
Feb 09, 2023 6:10:02 PM org.apache.pdfbox.pdmodel.font.PDTrueTypeFont <init>



,Lap Lap Time,T1,T2 T3
0,1st,93 Marc MARQUEZ,Repsol H
1,NaN,Runs=2,Total laps=1
2,1,2'35.705 59.846,33.384 32.567
3,2,2'07.150 35.689,31.435 31.390
4,3,2'04.783 35.263,30.210 30.824
5,4,2'05.254 35.407,30.300 30.924
6,5,2'04.355 34.977,30.357 30.585
7,6,2'04.182 35.071,29.930 30.735
8,7,2'13.741 P 37.175,32.292 32.355
9,8,7'38.066 5'58.255,33.123 36.095
